In [7]:
!pip install wandb

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split

In [9]:
import wandb
wandb.login(key="13b86763ab8ddf529c91c7dce385c6cb04b5253e")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m015 (iitm-ma23m015) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
from torchvision import transforms, datasets
train_set = "/kaggle/input/inature12k/inaturalist_12K/train"
test_set = "/kaggle/input/inature12k/inaturalist_12K/val"
IMG_SIZE = (224,224)
# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),        
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_set, transform=transform)
val_size = int(0.2 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])
testset = datasets.ImageFolder(root=test_set, transform=transform)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class pretrainedCNN():
    def __init__(self, trainset, valset, model, batch_size=32, freeze_percent=1):
        self.model = model

        # Freeze layers based on freeze_percent
        layers = list(self.model.parameters())
        num_layers_to_freeze = int(freeze_percent * len(layers))

        for i, param in enumerate(layers):
            param.requires_grad = False if i < num_layers_to_freeze else True

        # Replace the final FC layer
        num_classes = 10  
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        self.model.fc.to(device)

      
        self.dataloader_train = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
        self.dataloader_val = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

    def train(self, epochs=10, lr=0.001, weight_decay=0):
        self.model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.model.parameters()),
                               lr=lr, weight_decay=weight_decay)

        for epoch in range(1, epochs + 1):
            self.model.train()
            train_accuracy, train_loss = 0, 0
            total_train = 0

            for inputs, labels in self.dataloader_train:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * inputs.size(0)
                train_accuracy += (outputs.argmax(1) == labels).sum().item()
                total_train += labels.size(0)

            train_acc = train_accuracy / total_train
            wandb.log({'epoch': epoch, 'train_accuracy': train_acc * 100})

            # Validation
            self.model.eval()
            val_accuracy, val_loss = 0, 0
            total_val = 0

            with torch.no_grad():
                for inputs, labels in self.dataloader_val:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item() * inputs.size(0)
                    val_accuracy += (outputs.argmax(1) == labels).sum().item()
                    total_val += labels.size(0)

            val_acc = val_accuracy / total_val
            avg_val_loss = val_loss / total_val

            print(f"Epoch {epoch}: Train Acc: {train_acc*100:.2f}%  Val Acc: {val_acc*100:.2f}%")
            wandb.log({'epoch': epoch,
                       'validation_accuracy': val_acc * 100,
                       'validation_loss': avg_val_loss})





In [13]:
# Sweep config
sweep_config = {
    'method': 'bayes',
    'name': 'Pretuning sweep',
    'metric': {'goal': 'maximize', 'name': 'validation_accuracy'},
    'parameters': {
        'learning_rate': {'values': [1e-3, 1e-4]},
        'freeze_percent': {'values': [0.2, 0.6, 0.8, 0.9]},
        'L2_regularisation': {'values': [0, 0.0005, 0.05]},
        'batch_size': {'values': [32, 64]},
        'epochs': {'values': [10, 15]}
    }
}


In [14]:
sweep_id = wandb.sweep(sweep = sweep_config, project='DA6401-Assignment_2')

Create sweep with ID: qzj8r57u
Sweep URL: https://wandb.ai/iitm-ma23m015/DA6401-Assignment_2/sweeps/qzj8r57u


In [15]:
def main():
    with wandb.init() as run:
        config = wandb.config
        wandb.run.name = f"-bs_{config.batch_size}-ep_{config.epochs}-lr_{config.learning_rate}-freeze_{config.freeze_percent}"

        model = models.resnet50(pretrained=True)
        cnn_model = pretrainedCNN(trainset, valset, model,
                                  batch_size=config.batch_size,
                                  freeze_percent=config.freeze_percent)
        cnn_model.train(epochs=config.epochs,
                        lr=config.learning_rate,
                        weight_decay=config.L2_regularisation)

wandb.agent(sweep_id, function=main, count=2)
wandb.finish()

wandb: Agent Starting Run: wsn0tqp3 with config:
wandb: 	L2_regularisation: 0.05
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	freeze_percent: 0.6
wandb: 	learning_rate: 0.0001
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 204MB/s]


Epoch 1: Train Acc: 68.23% /n Val Acc: 69.88%
Epoch 2: Train Acc: 76.14% /n Val Acc: 62.58%
Epoch 3: Train Acc: 78.49% /n Val Acc: 67.33%
Epoch 4: Train Acc: 80.21% /n Val Acc: 67.83%
Epoch 5: Train Acc: 83.09% /n Val Acc: 69.43%
Epoch 6: Train Acc: 84.36% /n Val Acc: 66.68%
Epoch 7: Train Acc: 86.33% /n Val Acc: 62.98%
Epoch 8: Train Acc: 87.85% /n Val Acc: 68.93%
Epoch 9: Train Acc: 89.58% /n Val Acc: 68.43%
Epoch 10: Train Acc: 90.48% /n Val Acc: 66.73%


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_accuracy,▁▃▄▅▆▆▇▇██
validation_accuracy,█▁▆▆█▅▁▇▇▅
validation_loss,▁█▅▄▂▆█▃▄▇
epoch,10
train_accuracy,90.475
validation_accuracy,66.73337
validation_loss,1.12097


wandb: Agent Starting Run: p1ah1m7h with config:
wandb: 	L2_regularisation: 0
wandb: 	batch_size: 32
wandb: 	epochs: 15
wandb: 	freeze_percent: 0.6
wandb: 	learning_rate: 0.0001


Epoch 1: Train Acc: 70.17% /n Val Acc: 76.09%
Epoch 2: Train Acc: 88.64% /n Val Acc: 77.79%
Epoch 3: Train Acc: 95.14% /n Val Acc: 77.44%
Epoch 4: Train Acc: 96.89% /n Val Acc: 77.84%
Epoch 5: Train Acc: 98.02% /n Val Acc: 77.09%
Epoch 6: Train Acc: 98.00% /n Val Acc: 74.54%
Epoch 7: Train Acc: 97.69% /n Val Acc: 75.09%


wandb: Ctrl + C detected. Stopping sweep.


epoch,▁▁▂▂▃▃▅▅▆▆▇▇██
train_accuracy,▁▆▇████
validation_accuracy,▄█▇█▆▁▂
validation_loss,▁▁▃▃▆▇█
epoch,7
train_accuracy,97.6875
validation_accuracy,75.08754
validation_loss,1.05729
